## Wikidata API Example

This notebook demonstrates the usage of the Mediawiki API to get and save information on Wikidata.

* Create a new bot username and password: https://www.wikidata.org/wiki/Special:BotPasswords
* Create a file env.txt to save the values like that:
```
BOT_NAME=Metaodi@WikidataTrainingBot
BOT_PASSWORD=lkjdlkjjhoihnfjohshakklks
```

In [26]:
# install the dependencies
!pip install requests python-dotenv

In [27]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import datetime

# load environment variables from .env files (to store sensitive information like passwords)
load_dotenv(dotenv_path='env.txt')

session = requests.Session()

## Request an item from the API

In [28]:
WIKIDATA_API_URL = 'https://www.wikidata.org/w/api.php'

# sandbox item
item = 'Q15397819'

In [29]:
res = session.get(WIKIDATA_API_URL, params={
    'action': 'wbgetentities',
    'ids': item,
    'format': 'json',
})
result = res.json()
result

{'entities': {'Q15397819': {'pageid': 17254039,
   'ns': 0,
   'title': 'Q15397819',
   'lastrevid': 1043569022,
   'modified': '2019-11-02T20:13:14Z',
   'type': 'item',
   'id': 'Q15397819',
   'labels': {'ko': {'language': 'ko', 'value': '세 번째 연습장'},
    'en': {'language': 'en', 'value': 'Wikidata Sandbox 3'},
    'en-ca': {'language': 'en-ca', 'value': 'Third sandbox'},
    'el': {'language': 'el', 'value': 'Τρίτο Πρόχειρο'},
    'ru': {'language': 'ru', 'value': 'Песочница: Эпизод III'},
    'zh-hans': {'language': 'zh-hans', 'value': '维基数据沙盒3'},
    'mk': {'language': 'mk', 'value': 'Трет песочник'},
    'eo': {'language': 'eo', 'value': 'tria provejo'},
    'nl': {'language': 'nl', 'value': 'derde zandbak'},
    'cs': {'language': 'cs', 'value': 'třetí pískoviště'},
    'it': {'language': 'it', 'value': 'terza pagina delle prove'},
    'sv': {'language': 'sv', 'value': 'Tredje sandlådan'},
    'ja': {'language': 'ja', 'value': 'ウィキデータ・サンドボックス3'},
    'fr': {'language': 'fr', 'va

## Login to Wikidata

In [30]:
# login to wikidata
# Note: the login is saved in the requests session (i.e. in the cookies)
#       so make sure to use the same session for all subsequent calls
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'login',
    'format': 'json',
})
tokens = res.json()['query']['tokens']

res = session.post(WIKIDATA_API_URL, data={
    'action': 'login',
    'lgname': os.getenv('BOT_NAME'),
    'lgpassword': os.getenv('BOT_PASSWORD'),
    'lgtoken': tokens['logintoken'],
    'format': 'json'
})
login = res.json()
login

{'login': {'result': 'Success', 'lguserid': 733408, 'lgusername': 'Metaodi'}}

## Update the description of an item (+CSRF)

In [31]:
# generate csrf token
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'csrf',
    'format': 'json',
})
csrf = res.json()['query']['tokens']['csrftoken']
csrf

now = datetime.datetime.now()
# update description of sandbox item
res = session.post(WIKIDATA_API_URL, data={
    'action': 'wbsetdescription',
    'id': item,
    'token': csrf,
    'language': 'en',
    'value': 'My super wikidata bot, current time: %s' % now.isoformat(),
    'format': 'json',
})
res.json()

{'entity': {'descriptions': {'en': {'language': 'en',
    'value': 'My super wikidata bot, current time: 2019-11-02T20:14:42.285335'}},
  'id': 'Q15397819',
  'type': 'item',
  'lastrevid': 1043569760},
 'success': 1}

In [32]:
# request the item again
res = session.get(WIKIDATA_API_URL, params={
    'action': 'wbgetentities',
    'ids': item,
    'format': 'json',
})
result = res.json()
result['entities']['Q15397819']['descriptions']['en']

{'language': 'en',
 'value': 'My super wikidata bot, current time: 2019-11-02T20:14:42.285335'}

## Let's add a triple

To add a triple, we use the [`wbcreateclaim`](https://www.wikidata.org/w/api.php?action=help&modules=wbcreateclaim) action. Let's see what parameters are needed.

In [37]:
# generate csrf token
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'csrf',
    'format': 'json',
})
csrf = res.json()['query']['tokens']['csrftoken']
csrf

# add triple to item
res = session.post(WIKIDATA_API_URL, data={
    'action': 'wbcreateclaim',
    'entity': item,
    'token': csrf,
    'snaktype': 'value',
    'property': 'P106',
    'value': {'entity-type': 'item', 'numeric-id': 212238},
    'format': 'json',
})
res.json()

{'error': {'code': 'invalid-snak',
  'info': 'Invalid snak data.',
  'messages': [{'name': 'wikibase-api-invalid-snak',
    'parameters': [],
    'html': {'*': 'Invalid snak data.'}}],
  '*': 'See https://www.wikidata.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'},
 'servedby': 'mw1315'}